In [1]:
import json
import string
import re
import nltk
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_set = open("C:/Users/user/Desktop/ML_Challenge/train-1.json")
test_set = open("C:/Users/user/Desktop/ML_Challenge/test.json")
train = json.load(train_set)
test = json.load(test_set)

In [3]:
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

In [4]:
df_train.head()

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"[A. Akbik, Laura Chiticariu, Marina Danilevsky...",ACL,2015.0,39,[Semantic role labeling],True,[Computer Science],60
1,10.18653/v1/2020.eval4nlp-1.12,One of these words is not like the other: a re...,Word embeddings are an active topic in the NLP...,"[Jesper Brink Andersen, Mikkel Bak Bertelsen, ...",EVAL4NLP,2020.0,44,[],True,[Computer Science],1
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"[Kyle Richardson, Sina Zarrieß, Jonas Kuhn]",INLG,2017.0,30,"[Natural language generation, Library (computi...",True,[Computer Science],5
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"[Rik van Noord, Johan Bos]",SemEval@ACL,2017.0,11,"[Parsing, Convolutional neural network, Text-b...",True,[Computer Science],5
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"[Elisabeth Lien, Milen Kouylekov]",IWPT,2015.0,26,"[Textual entailment, Parsing, SemEval, Semanti...",True,[Computer Science],10


In [5]:
df_train.shape

(9658, 11)

In [6]:
df_train.isnull().sum()

doi                  0
title                0
abstract           159
authors              0
venue                0
year                 3
references           0
topics               0
is_open_access       0
fields_of_study    136
citations            0
dtype: int64

In [7]:
df_train = df_train.dropna()

In [8]:
df_train["authors"] = [",".join(i) for i in df_train["authors"]]
df_train["topics"] = [",".join(i) for i in df_train["topics"]]
df_train["fields_of_study"] = [",".join(i) for i in df_train["fields_of_study"]]

In [9]:
df_train = df_train.replace(r'^\s*$', np.nan, regex=True)

In [10]:
df_train.isnull().sum()

doi                   0
title                 0
abstract              0
authors               0
venue               403
year                  0
references            0
topics             1940
is_open_access        0
fields_of_study       0
citations             0
dtype: int64

In [11]:
df_train = df_train.dropna()

In [12]:
#mode_venue = df_train["venue"].mode().values[0]
#mode_venue

In [13]:
#df_train["venue"] = df_train["venue"].fillna(mode_venue)

In [14]:
def clean_venue(text):
    n = re.sub(r'[0-9]', "", text)
    p = re.sub(r'[^\w\s]', "", n.lower().strip())
    return p

In [15]:
df_train["venue"] = df_train["venue"].apply(lambda x: clean_venue(x))

In [16]:
df_train['venue_rank'] = df_train.groupby('venue')['citations'].rank(pct=True, ascending = False)

In [17]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,venue_rank
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"A. Akbik,Laura Chiticariu,Marina Danilevsky,Yu...",acl,2015.0,39,Semantic role labeling,True,Computer Science,60,0.232098
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"Kyle Richardson,Sina Zarrieß,Jonas Kuhn",inlg,2017.0,30,"Natural language generation,Library (computing...",True,Computer Science,5,0.556522
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"Rik van Noord,Johan Bos",semevalacl,2017.0,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5,0.558824
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"Elisabeth Lien,Milen Kouylekov",iwpt,2015.0,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10,0.187500
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"Xian Qian,Yang Liu",conll shared task,2017.0,12,"Feature engineering,Parsing,Linear classifier,...",True,Computer Science,4,0.704545
...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"Hongge Chen,Huan Zhang,Pin-Yu Chen,Jinfeng Yi,...",acl,2018.0,44,"Adversary (cryptography),Visual language",True,Computer Science,72,0.193245
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"Neha Nayak Kennard,Gabor Angeli,Christopher D....",repevalacl,2016.0,15,"Word embedding,Microsoft Word for Mac,Natural ...",True,Computer Science,49,0.300000
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"M. Hasan,Wasifur Rahman,Amir Zadeh,Jianyuan Zh...",emnlp,2019.0,52,"Multimodal interaction,Natural language proces...",True,"Computer Science,Mathematics",34,0.344731
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"Rudolf Kadlec,Jindřich Libovický,Jan Macek,Jan...",dmeacl,2014.0,25,"Dialog system,Discriminative model,Inclusion B...",True,Computer Science,8,0.111111


In [18]:
X = df_train[["venue_rank"]]

In [19]:
y = df_train[["citations"]]

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [22]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)

In [23]:
def score(Y_true, Y_pred):
    y_true = np.log1p(np.maximum(0, Y_true))
    y_pred = np.log1p(np.maximum(0, Y_pred))
    return 1 - np.mean((y_true-y_pred)**2) / np.mean((y_true-np.mean(y_true))**2)

def evaluate(gold_path, pred_path):
    gold = { x['doi']: x['citations'] for x in json.load(open(gold_path)) }
    pred = { x['doi']: x['citations'] for x in json.load(open(pred_path)) }
    y_true = np.array([ gold[key] for key in gold ])
    y_pred = np.array([ pred[key] for key in gold ])
    return score(y_true, y_pred)

In [24]:
score(y_val, y_pred)

citations    0.156218
dtype: float64

In [25]:
from sklearn.metrics import r2_score

r2_score(y_val, y_pred)

0.0924977438819824

In [26]:
df_train["year"] = df_train["year"].astype(int)

In [27]:
df_train["age_of_article"] = [2021 - i for i in df_train['year']]

In [28]:
authors = df_train[["authors"]]
authors['authors_k'] = authors['authors'].apply(lambda x: " ".join(x.lower() for x in x.split()))
authors['authors_p'] = [x.replace("."," ").split(',') for x in authors['authors_k']]
df_train["authors"] = authors['authors_p']

In [29]:
df_train["authors"] = [",".join(i) for i in df_train["authors"]]

In [30]:
from collections import Counter
def split_and_count(column):
    lis = []
    for i in column:
        for j in i.split(","):
            lis.append(j)
            counts = Counter(lis)
    return counts

In [31]:
df_train["author_counts"] = df_train['authors'].apply(lambda x: len(x.split(',')))

In [32]:
df = pd.DataFrame(list(split_and_count(df_train["authors"]).items()),columns = ['authors','counts'])

In [33]:
def author_score(var1,var2,main_df,index): 
    k = 0
    for i,j in zip(var1,var2):
        if i in main_df[index]:
            k += j
    return k

In [34]:
df_train["author_score"] = [author_score(df["authors"], df["counts"],df_train["authors"], each) for each in df_train.index]

In [35]:
df_train['author_rank'] = df_train.groupby('author_score')['citations'].rank(pct=True, ascending = True)

In [36]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,venue_rank,age_of_article,author_counts,author_score,author_rank
0,10.3115/v1/P15-1039,Generating High Quality Proposition Banks for ...,Semantic role labeling (SRL) is crucial to nat...,"a akbik,laura chiticariu,marina danilevsky,yu...",acl,2015,39,Semantic role labeling,True,Computer Science,60,0.232098,6,6,14,0.879397
2,10.18653/v1/W17-3516,The Code2Text Challenge: Text Generation in So...,We propose a new shared task for tactical data...,"kyle richardson,sina zarrieß,jonas kuhn",inlg,2017,30,"Natural language generation,Library (computing...",True,Computer Science,5,0.556522,4,3,21,0.200000
3,10.18653/v1/S17-2160,The Meaning Factory at SemEval-2017 Task 9: Pr...,We evaluate a semantic parser based on a chara...,"rik van noord,johan bos",semevalacl,2017,11,"Parsing,Convolutional neural network,Text-base...",True,Computer Science,5,0.558824,4,2,8,0.280000
4,10.18653/v1/W15-2205,Semantic Parsing for Textual Entailment,In this paper we gauge the utility of general-...,"elisabeth lien,milen kouylekov",iwpt,2015,26,"Textual entailment,Parsing,SemEval,Semantic We...",True,Computer Science,10,0.187500,6,2,5,0.617560
5,10.18653/v1/K17-3015,A non-DNN Feature Engineering Approach to Depe...,For this year’s multilingual dependency parsin...,"xian qian,yang liu",conll shared task,2017,12,"Feature engineering,Parsing,Linear classifier,...",True,Computer Science,4,0.704545,4,2,27,0.159091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,Attacking Visual Language Grounding with Adver...,Visual language grounding is widely studied in...,"hongge chen,huan zhang,pin-yu chen,jinfeng yi,...",acl,2018,44,"Adversary (cryptography),Visual language",True,Computer Science,72,0.193245,3,5,8,0.908000
9650,10.18653/v1/W16-2504,Evaluating Word Embeddings Using a Representat...,Word embeddings are increasingly used in natur...,"neha nayak kennard,gabor angeli,christopher d ...",repevalacl,2016,15,"Word embedding,Microsoft Word for Mac,Natural ...",True,Computer Science,49,0.300000,5,3,37,0.580000
9651,10.18653/v1/D19-1211,UR-FUNNY: A Multimodal Language Dataset for Un...,Humor is a unique and creative communicative b...,"m hasan,wasifur rahman,amir zadeh,jianyuan zh...",emnlp,2019,52,"Multimodal interaction,Natural language proces...",True,"Computer Science,Mathematics",34,0.344731,2,7,18,0.702454
9653,10.3115/v1/W14-0202,IBM’s Belief Tracker: Results On Dialog State ...,Accurate dialog state tracking is crucial for ...,"rudolf kadlec,jindřich libovický,jan macek,jan...",dmeacl,2014,25,"Dialog system,Discriminative model,Inclusion B...",True,Computer Science,8,0.111111,7,4,10,0.451754


In [37]:
#topic popularity/ranking based on average citation count

In [38]:
#age of an article/number of years since the article was published

In [39]:
X_author = df_train[["author_rank", "venue_rank"]]
y_author = df_train[["citations"]]

In [40]:
X_t, X_v, y_t, y_v = train_test_split(X_author, y_author, test_size=0.2, random_state=42)

In [41]:
ss = StandardScaler()

X_t = ss.fit_transform(X_t)
X_v = ss.transform(X_v)

In [42]:
linreg = LinearRegression()

linreg.fit(X_t,y_t)

y_pred_auth = linreg.predict(X_v)

In [43]:
score(y_v, y_pred_auth)

citations    0.312915
dtype: float64

In [44]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
Stopwords = set(stopwords.words('english'))

In [45]:
def clean_text(text):
    text_clean = "".join([word.lower() for word in text if word not in string.punctuation])
    text_rc = re.sub('[0-9]+', '', text_clean)
    tokens = re.split('\W+', text_rc)
    text = [word for word in tokens if word not in Stopwords]
    return text

In [46]:
df_train["abstract"] = df_train["abstract"].apply(lambda x: clean_text(x))

In [70]:
df_train["abstract"] = [" ".join(i) for i in df_train["abstract"]]

In [48]:
df_train["title"] = df_train["title"].apply(lambda x: clean_text(x))

In [102]:
df_train["title"] = [" ".join(i) for i in df_train["title"]]

In [50]:
topics = df_train[["topics"]]
topics['topics_k'] = topics['topics'].apply(lambda x: " ".join(x.lower() for x in x.split()))
topics['topics_p'] = [x.replace(" ", ",").replace("(","").replace(")","").split(',') for x in topics['topics_k']]
df_train["topics"] = topics["topics_p"]

In [110]:
df_train["topics"] = [" ".join(i) for i in df_train["topics"]]

In [103]:
df_train

,doi,title,abstract,authors,venue,year,references,topics,is_open_access,fields_of_study,citations,venue_rank,age_of_article,author_counts,author_score,author_rank
0,10.3115/v1/P15-1039,generating high quality proposition banks mult...,semantic role labeling srl crucial natural lan...,"a akbik,laura chiticariu,marina danilevsky,yu...",acl,2015,39,"[semantic, role, labeling]",True,Computer Science,60,0.232098,6,6,14,0.879397
2,10.18653/v1/W17-3516,codetext challenge text generation source libr...,propose new shared task tactical datatotext ge...,"kyle richardson,sina zarrieß,jonas kuhn",inlg,2017,30,"[natural, language, generation, library, compu...",True,Computer Science,5,0.556522,4,3,21,0.200000
3,10.18653/v1/S17-2160,meaning factory semeval task producing amrs ne...,evaluate semantic parser based characterbased ...,"rik van noord,johan bos",semevalacl,2017,11,"[parsing, convolutional, neural, network, text...",True,Computer Science,5,0.558824,4,2,8,0.280000
4,10.18653/v1/W15-2205,semantic parsing textual entailment,paper gauge utility generalpurpose opendomain ...,"elisabeth lien,milen kouylekov",iwpt,2015,26,"[textual, entailment, parsing, semeval, semant...",True,Computer Science,10,0.187500,6,2,5,0.617560
5,10.18653/v1/K17-3015,nondnn feature engineering approach dependency...,year multilingual dependency parsing shared ta...,"xian qian,yang liu",conll shared task,2017,12,"[feature, engineering, parsing, linear, classi...",True,Computer Science,4,0.704545,4,2,27,0.159091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,10.18653/v1/P18-1241,attacking visual language grounding adversaria...,visual language grounding widely studied moder...,"hongge chen,huan zhang,pin-yu chen,jinfeng yi,...",acl,2018,44,"[adversary, cryptography, visual, language]",True,Computer Science,72,0.193245,3,5,8,0.908000
9650,10.18653/v1/W16-2504,evaluating word embeddings using representativ...,word embeddings increasingly used natural lang...,"neha nayak kennard,gabor angeli,christopher d ...",repevalacl,2016,15,"[word, embedding, microsoft, word, for, mac, n...",True,Computer Science,49,0.300000,5,3,37,0.580000
9651,10.18653/v1/D19-1211,urfunny multimodal language dataset understand...,humor unique creative communicative behavior o...,"m hasan,wasifur rahman,amir zadeh,jianyuan zh...",emnlp,2019,52,"[multimodal, interaction, natural, language, p...",True,"Computer Science,Mathematics",34,0.344731,2,7,18,0.702454
9653,10.3115/v1/W14-0202,ibm belief tracker results dialog state tracki...,accurate dialog state tracking crucial design ...,"rudolf kadlec,jindřich libovický,jan macek,jan...",dmeacl,2014,25,"[dialog, system, discriminative, model, inclus...",True,Computer Science,8,0.111111,7,4,10,0.451754


In [80]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [83]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [104]:
abstract = df_train[["abstract"]]
title = df_train[["title"]]

In [105]:
lemmatizer = WordNetLemmatizer()

wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

abstract["2nd_lemmatized"] = abstract["abstract"].apply(lambda text: lemmatize_words(text))
title["title_lem"] = title["title"].apply(lambda text: lemmatize_words(text))

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [101]:
vectorizer = TfidfVectorizer(analyzer='word', 
                              token_pattern=r'\b[a-zA-Z]{3,}\b',
                              max_features=200, stop_words = ["also", "two", "use", "one", "without"])  
vectorized = vectorizer.fit_transform(abstract["2nd_lemmatized"])
pd.DataFrame(vectorized.toarray(),columns=vectorizer.get_feature_names())

,accuracy,achieve,across,address,aim,algorithm,allow,analysis,annotate,annotation,...,tree,type,unsupervised,user,various,vector,way,well,word,work
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0.131021,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,0.000000,0.038765,0.0,0.0,0.0,0.000000,0.000000,0.039513,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.036086
4,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.113698,0.0,0.0,...,0.481226,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7094,0.000000,0.000000,0.0,0.0,0.0,0.355985,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
7095,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.132793,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.256945,0.000000
7096,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.096703,0.000000
7097,0.217411,0.000000,0.0,0.0,0.0,0.000000,0.124346,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.119908,0.0,0.0,0.0,0.096165,0.000000,0.000000


In [109]:
vectorizer1 = TfidfVectorizer(analyzer='word', 
                              token_pattern=r'\b[a-zA-Z]{3,}\b',
                              max_features=100, stop_words = ["also", "two", "use", "one", "without", "via", "wmt"])  

vectorized1 = vectorizer1.fit_transform(title["title_lem"])
pd.DataFrame(vectorized1.toarray(),columns=vectorizer1.get_feature_names()).columns

Index(['alignment', 'analysis', 'annotation', 'answer', 'approach', 'arabic',
       'attention', 'automatic', 'base', 'case', 'chinese', 'classification',
       'comprehension', 'context', 'corpus', 'crosslingual', 'data', 'deep',
       'dependency', 'detect', 'detection', 'dialog', 'dialogue',
       'disambiguation', 'discourse', 'document', 'domain', 'embeddings',
       'emotion', 'english', 'entity', 'error', 'evaluation', 'event',
       'extraction', 'feature', 'framework', 'generate', 'generation',
       'grammar', 'graph', 'identification', 'identify', 'improve',
       'information', 'joint', 'knowledge', 'label', 'language', 'learn',
       'learning', 'lexical', 'linguistic', 'machine', 'method', 'model',
       'morphological', 'multilingual', 'multimodal', 'name', 'natural',
       'network', 'neural', 'news', 'nlp', 'online', 'parse', 'parsing',
       'predict', 'prediction', 'question', 'recognition', 'relation',
       'representation', 'segmentation', 'semantic',